In [230]:
import pandas as pd
import folium
from IPython.core.display import display
import numpy as np
from folium.plugins import TimestampedGeoJson
from selenium import webdriver
from datetime import datetime
import json
import re
import fileinput




In [166]:
# IDEAS:
# for the world we could change color by city / country or use color densities (see links below in the code for implementation)
# add colorscale to neighborhoods, for example density of some parameter

In [203]:
# USER SETTINGS:

# use the detailed listings.csv so that first review dates are present
input_csv_file = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/2019-09-14_Amsterdam_listings_detailed.csv'
# to display neighbourhoods
geojson_file = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/NaT_Amsterdam_neighbourhoods.geojson'
# all the names of the neighbourhoods
neighbourhoods_file = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/NaT_Amsterdam_neighbourhoods.csv'
# output .html maps saving destination
saving_path = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Outputs'
# used later for display, can be a city or the word
airbnb_location = 'Amsterdam'
# show Neighbourhoods areas on the map
displayNeighbourhoods = True

In [193]:
if displayNeighbourhoods:
    with open(geojson_file) as f:
        jsonFile = json.load(f)
    neighbourhoodsLabels = pd.read_csv(neighbourhoods_file)


df = pd.read_csv(input_csv_file, low_memory = False);

movieData = df[['first_review', 'latitude', 'longitude']]
movieData = movieData.dropna()
movieData.first_review.asty = movieData.first_review.astype(str)
movieData['month'] = movieData.first_review.apply(lambda x: x.split('-')[1]).astype('int64')
movieData['year'] = movieData.first_review.apply(lambda x: x.split('-')[0]).astype('int64')
movieData = movieData.sort_values(by = 'year', ascending = True)

grouped = movieData.groupby(['year', 'month'])

In [204]:
neighbourhoodsLabels = pd.read_csv(neighbourhoods_file)

,neighbourhood_group,neighbourhood
0,NaN,Bijlmer-Centrum
1,NaN,Bijlmer-Oost
2,NaN,Bos en Lommer
3,NaN,Buitenveldert - Zuidas
4,NaN,Centrum-Oost
5,NaN,Centrum-West
6,NaN,De Aker - Nieuw Sloten
7,NaN,De Baarsjes - Oud-West
8,NaN,De Pijp - Rivierenbuurt
9,NaN,Gaasperdam - Driemond


In [ ]:
'''
for _, data in grouped:
    print('----->')
    print(data)
    print(type(data))
    print('LATITUDE')
    print(data.latitude)
    print('')
    for i in data:
        print(type(i))
'''

In [273]:
# inspiration: 
# https://towardsdatascience.com/visualizing-air-pollution-with-folium-maps-4ce1a1880677
# also look this:
# https://towardsdatascience.com/visualizing-bike-mobility-in-london-using-interactive-maps-for-absolute-beginners-3b9f55ccb59
# https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON
# technical docs:
# https://python-visualization.github.io/folium/plugins.html
# https://python-visualization.github.io/folium/plugins.html
# cart styles:
# https://deparkes.co.uk/2016/06/10/folium-map-tiles/
# add like this in the code below: folium.Map(location = London, zoom_start = 12, tiles = "CartoDB dark_matter") 
# https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data

# geojson visualization style:
# https://stackoverflow.com/questions/44919669/folium-geojson-style-function-not-working-as-i-want

# use TimestampedGeoJson to animate the folium map. It requires a geojson input.
# create_geojson_features converts df into geojson

# add popups:
# https://www.jpytr.com/post/analysinggeographicdatawithfolium/
# TO OPEN POPUPS IN MOUSEOVER INSTEAD ON CLICKS:
# https://stackoverflow.com/questions/41095716/hover-in-popup-in-folium

def create_geojson_features(groupedDF):
    print('> Creating GeoJSON features...')
    features = []
    for _, data in grouped:
        for _, row in data.iterrows():  
            date = str(row['year']) + '-' + str(row['month'])
            #date = datetime.date(year=row['year'], month=row['month'])
            #date = str(date.year) + '-' + str(date.month)
            # keep only date with date() (remove time) and only YYYY/MM (remove DD)
            date = datetime.strptime(date, '%Y-%m').date().__str__()[0:-3]
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type':'Point', 
                    'coordinates':[row['longitude'],row['latitude']]
                },
                'properties': {
                    # 'time': row['DatetimeBegin'].date().__str__(),
                    'time': date,
                    'icon': 'circle',
                    'iconstyle':{
                        'color': 'magenta',
                        'fillOpacity': 0.8,
                        'stroke': 'true',
                        'radius': 2
                    }
                }
            }
        features.append(feature)
    return features



def make_map(features, coord_start):
    print('> Making map...')
    airbnb_map = folium.Map(location = coord_start, control_scale = True, zoom_start = 11)

    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period = 'P1M'
        , add_last_point = True
        , auto_play = False
        , loop = False
        , max_speed = 5
        , loop_button = True
        , date_options = 'YYYY/MM'
        , time_slider_drag_update = True
    ).add_to(airbnb_map)
    
    if displayNeighbourhoods:
        layer_gson = folium.FeatureGroup(name='geojson')

        for i, geo_json in enumerate(jsonFile['features']):
            #print(geo_json)
            gj = folium.GeoJson(
                geo_json,
                name='geojson',
                style_function=lambda feature: {
                    'fillColor': 'yellow',
                    'color' : 'blue',
                    'weight' : 1,
                    'fillOpacity' : 0.2,
                    }
                )
            popup = folium.Popup(neighbourhoodsLabels.neighbourhood[i])
            gj.add_child(popup)
            gj.add_to(layer_gson)
            #airbnb_map.add_child(gj)
        
        layer_gson.add_to(airbnb_map)
        folium.LayerControl().add_to(airbnb_map)
        #folium.LayerControl().add_to(airbnb_map)
        
        '''
        # add neighbourhood without popups
        folium.GeoJson(
            jsonFile,
            name='geojson',
            style_function=lambda feature: {
                'fillColor': 'yellow',
                'color' : 'blue',
                'weight' : 1,
                'fillOpacity' : 0.2,
                }
            ).add_to(airbnb_map)
        '''
        
        
    print('> Done')
    return airbnb_map

def plot_airbnb(location, groupedDF, coord_start):
    print('Mapping {} AIRBNB'.format(location))
    # df = load_data(pollutant_ID)
    # df = clean_data(df)
    # df = prepare_data(df, pollutant_ID)
    features = create_geojson_features(groupedDF)
    return make_map(features, coord_start)

In [ ]:
'''
features = create_geojson_features(grouped)
coord_start = [movieData.longitude.mean(), movieData.latitude.mean()]
mm = make_map(features, coord_start)
'''

In [274]:
coord_start = [movieData.latitude.mean(), movieData.longitude.mean()]
airbnb_map = plot_airbnb(airbnb_location, grouped, coord_start)

# save map
if displayNeighbourhoods:
    saving_name = saving_path + '/' + airbnb_location + '_with_neighborhoods.html'
else:
    saving_name = saving_path + '/' + airbnb_location + '.html'
    
airbnb_map.save(saving_name)

# display map
airbnb_map

Mapping Amsterdam AIRBNB
> Creating GeoJSON features...
> Making map...
> Done


In [ ]:
# DISPLAY POPUP OVER_MOUSE (THIS CODE DOESN'T WORK)

saved_map = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Outputs/Amsterdam_with_neighborhoods.html'
with open(saved_map) as inf:
    txt = inf.read()

#Find all the markers names given by folium
markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))

for linenum,line in enumerate( fileinput.FileInput(saved_map,inplace=1) ):
    pattern = markers[0] + ".bindPopup"
    pattern2 = markers[0] + ".on('mouseover', function (e) {this.openPopup();});"
    pattern3 = markers[0] + ".on('mouseout', function (e) {this.closePopup();});"

    if pattern in line:
        print(line.rstrip())
        print(pattern2)
        print(pattern3)
    else:
        print(line.rstrip())
        
